In [1]:
%matplotlib inline
import os, sys
#sys.path.append('/Users/hhsieh/anaconda3/envs/astroconda/lib/python3.6/site-packages')
#sys.path.append('/opt/local/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages')
print(sys.version)
print('')
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.io.fits import getheader
from astropy.modeling import models
from astropy import nddata
from astropy import units as u
import ccdproc as cp
from ccdproc import CCDData, ImageFileCollection
import numpy as np
import glob, os, bz2, subprocess
import sqlite3
from sqlite3 import Error
import datetime


3.7.16 (default, Jan 17 2023, 09:28:58) 
[Clang 14.0.6 ]



In [2]:
def create_directory(path):
    if not os.path.exists(path):
        os.mkdir(path)
        print('Directory {:s} successfully created.'.format(path))
    else:
        print('Directory {:s} already exists.'.format(path))
    return None

def decompress_file_bzip2(filename):
    cmd = ['bzip2','-d',filename]
    process = subprocess.call(cmd)
    return None

def compress_file_fpack(filename):
    fpack_command  = '/Users/hhsieh/Astro/tools/cfitsio/fpack'
    delete_command = '/bin/rm'
    fpack_cmd = [fpack_command,filename]
    process = subprocess.call(fpack_cmd)
    delete_cmd = [delete_command,filename]
    process = subprocess.call(delete_cmd)
    return None

def decompress_file_fpack(filename):
    funpack_command = '/Users/hhsieh/Astro/tools/cfitsio/funpack'
    delete_command  = '/bin/rm'
    funpack_cmd = [funpack_command,filename]
    process = subprocess.call(funpack_cmd)
    delete_cmd = [delete_command,filename]
    process = subprocess.call(delete_cmd)
    return None

def decompress_directory_bzip2(file_path):
    print('>>> Starting decompression of bz2 files in {:s}...'.format(file_path))
    os.chdir(file_path)
    for bz2_file in glob.glob('*.bz2'):
        decompress_file_bzip2(bz2_file)
        #cmd = [bzip1,bzip2,bz2_file]
        #process = subprocess.call(cmd)
    print('>>> Decompression of bz2 files complete.')
    return None

def compress_directory_fpack(file_path):
    print('>>> Starting fpack compression of FITS files in {:s}...'.format(file_path))
    #fpack_file = '/Users/hhsieh/Astro/tools/cfitsio/fpack'
    #delete_file = '/bin/rm'
    os.chdir(file_path)
    for fits_file in glob.glob('*.fits'):
        compress_file_fpack(fits_file)
        #fpack_cmd = [fpack_file,fits_file]
        #process = subprocess.call(fpack_cmd)
        #delete_cmd = [delete_file,fits_file]
        #process = subprocess.call(delete_cmd)
    print('>>> fpack compression of FITS files complete.')
    return None
    
def decompress_directory_fpack(file_path):
    print('>>> Starting decompression of fz files in {:s}...'.format(file_path))
    #funpack_file = '/Users/hhsieh/Astro/tools/cfitsio/funpack'
    #delete_file = '/bin/rm'
    os.chdir(file_path)
    for fz_file in glob.glob('*.fz'):
        decompress_file_fpack(fz_file)
        #funpack_cmd = [funpack_file,fz_file]
        #process = subprocess.call(funpack_cmd)
        #delete_cmd = [delete_file,fz_file]
        #process = subprocess.call(delete_cmd)
    print('>>> Decompression of fz files complete.')
    return None


def get_date_id():
    # obtain YYYYMMDD date from filename of GMOS FITS files being processed
    # return: YYYYMMDD date id
    for raw_mef_file in glob.glob('*.fits'):
        fits_date = raw_mef_file[1:9]
    return fits_date


def flatfield_median_combine(flatfield_name,reduction_log):
    print('{:s} - Starting median combination of flatfield frames...'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')))
    reduction_log.write('{:s} - Starting median combination of flatfield frames...\n'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')))

    flatfield_list = []
    output_flat_filename = flatfield_name + '.chip1.fits'
    for fits_file in glob.glob('*.chip1.otz.fits'):
        hdulist = fits.open(fits_file)
        fits_data = hdulist[0].data
        fits_hdr = hdulist[0].header
        fits_mean = np.mean(fits_data)
        fits_data_n = fits_data / fits_mean
        file_prefix = fits_file[0:19]
        fits_file_n = file_prefix + '_n.otz.fits'
        fits.writeto(fits_file_n,fits_data_n,fits_hdr)
        hdulist.close()
        fits_data = CCDData.read(fits_file_n)
        flatfield_list.append(fits_data)
        delete_file = '/bin/rm'
        delete_cmd = [delete_file,fits_file_n]
        process = subprocess.call(delete_cmd)
    master_flat = cp.combine(flatfield_list,method='median')
    master_flat.write(output_flat_filename)
    print('>>> Median combination of flatfield frames for Chip 1 complete.')
    reduction_log.write('>>> Median combination of flatfield frames for Chip 1 complete.\n')

    flatfield_list = []
    output_flat_filename = flatfield_name + '.chip2.fits'
    for fits_file in glob.glob('*.chip2.otz.fits'):
        hdulist = fits.open(fits_file)
        fits_data = hdulist[0].data
        fits_hdr = hdulist[0].header
        #fits_mean = np.mean(fits_data)
        fits_mean = np.mean(fits_data[251:1012,1:2042])
        fits_data_n = fits_data / fits_mean
        file_prefix = fits_file[0:19]
        fits_file_n = file_prefix + '_n.otz.fits'
        fits.writeto(fits_file_n,fits_data_n,fits_hdr)
        hdulist.close()
        fits_data = CCDData.read(fits_file_n)
        flatfield_list.append(fits_data)
        delete_file = '/bin/rm'
        delete_cmd = [delete_file,fits_file_n]
        process = subprocess.call(delete_cmd)
    master_flat = cp.combine(flatfield_list,method='median')
    master_flat.write(output_flat_filename)
    print('>>> Median combination of flatfield frames for Chip 2 complete.')
    reduction_log.write('>>> Median combination of flatfield frames for Chip 2 complete.\n')

    flatfield_list = []
    output_flat_filename = flatfield_name + '.chip3.fits'
    for fits_file in glob.glob('*.chip3.otz.fits'):
        hdulist = fits.open(fits_file)
        fits_data = hdulist[0].data
        fits_hdr = hdulist[0].header
        fits_mean = np.mean(fits_data)
        fits_data_n = fits_data / fits_mean
        file_prefix = fits_file[0:19]
        fits_file_n = file_prefix + '_n.otz.fits'
        fits.writeto(fits_file_n,fits_data_n,fits_hdr)
        hdulist.close()
        fits_data = CCDData.read(fits_file_n)
        flatfield_list.append(fits_data)
        delete_file = '/bin/rm'
        delete_cmd = [delete_file,fits_file_n]
        process = subprocess.call(delete_cmd)
    master_flat = cp.combine(flatfield_list,method='median')
    master_flat.write(output_flat_filename)
    print('>>> Median combination of flatfield frames for Chip 3 complete.')
    reduction_log.write('>>> Median combination of flatfield frames for Chip 3 complete.\n')

    print('{:s} - Median combination of flatfield frames complete.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')))
    reduction_log.write('{:s} - Median combination of flatfield frames complete.\n'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')))

    return None

        
print('Done.')


Done.


In [3]:
def process_flatfield_directories(basewd):
    filter_id = basewd[-2:-1]
    instrname = 'not recognized'
    if basewd[-19:-18] == 'N': instrname = 'gmosn'
    if basewd[-19:-18] == 'S': instrname = 'gmoss'

    rawfits_dir  = '/rawfits_twiskyflat/'
    rawbias_dir  = '/rawfits_bias/'
    procfits_dir = '/procfits_twiskyflat/'
    exclfits_dir = '/procfits_twiskyflat/exclude/'

    os.chdir(basewd)
    for datadir in sorted(glob.glob('ut*_gemini?_twiskyflat')):
        path_rawfits  = basewd + datadir + rawfits_dir
        path_rawbias  = basewd + datadir + rawbias_dir
        path_procfits = basewd + datadir + procfits_dir
        path_exclfits = basewd + datadir + exclfits_dir
    
        flatfield_name = 'n' + datadir[2:10] + '.' + instrname + '.twiskyflat.' + filter_id
        flatfits_file_chip2 = path_procfits + flatfield_name + '.chip2.fits.fz'
        reduction_logfile = basewd + datadir + '/log_reduction_' + datadir[2:10] + '_' + instrname + '.txt'
    
        if not os.path.exists(flatfits_file_chip2):
            if os.path.exists(path_procfits):
                if os.path.exists(path_exclfits):
                    if instrname != 'not recognized':
                        with open(reduction_logfile,'w') as reduction_log:
                            print('\n{:s} - Starting median combination of flatfield files in {:s}...'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),datadir))
                            reduction_log.write('\n{:s} - Starting median combination of flatfield files in {:s}...\n'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),datadir))
                            os.chdir(path_procfits)
                            decompress_directory_fpack(path_procfits)                
                            flatfield_median_combine(flatfield_name,reduction_log)
                            compress_directory_fpack(path_procfits)     
                            print('{:s} - Processing flatfield files in {:s} complete.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),datadir))
                            reduction_log.write('\n{:s} - Processing flatfield files in {:s} complete.\n'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),datadir))
                    else:
                        print('{:s} - Instrument for {:s} not recognized.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),datadir))
                else:
                    print('{:s} - Flatfield files in {:s} not yet screened.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),datadir))
            else:
                print('{:s} - Flatfield files in {:s} not yet processed.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),datadir))
        else:
            print('{:s} - Median flatfields for {:s} already generated.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),datadir))

    return None


In [5]:
# Set base working directory and subdirectory paths

#process_flatfield_directories('/volumes/Fantom12a/BackupData/gemini/data_calib/GMOS-N_TwilightFlats_20170301_present/GMOS-N_Twilight_Flats_g/')
process_flatfield_directories('/volumes/Fantom12a/BackupData/gemini/data_calib/GMOS-N_TwilightFlats_20170301_present/GMOS-N_Twilight_Flats_r/')
#process_flatfield_directories('/volumes/Fantom12a/BackupData/gemini/data_calib/GMOS-N_TwilightFlats_20170301_present/GMOS-N_Twilight_Flats_i/')
#process_flatfield_directories('/volumes/Fantom12a/BackupData/gemini/data_calib/GMOS-N_TwilightFlats_20170301_present/GMOS-N_Twilight_Flats_z/')
#process_flatfield_directories('/volumes/Fantom12a/BackupData/gemini/data_calib/GMOS-S_TwilightFlats_20150101_present/GMOS-S_Twilight_Flats_u/')
#process_flatfield_directories('/volumes/Fantom12a/BackupData/gemini/data_calib/GMOS-S_TwilightFlats_20150101_present/GMOS-S_Twilight_Flats_g/')
process_flatfield_directories('/volumes/Fantom12a/BackupData/gemini/data_calib/GMOS-S_TwilightFlats_20150101_present/GMOS-S_Twilight_Flats_r/')
#process_flatfield_directories('/volumes/Fantom12a/BackupData/gemini/data_calib/GMOS-S_TwilightFlats_20150101_present/GMOS-S_Twilight_Flats_i/')
#process_flatfield_directories('/volumes/Fantom12a/BackupData/gemini/data_calib/GMOS-S_TwilightFlats_20150101_present/GMOS-S_Twilight_Flats_z/')

            
print('Done.')


2024-04-04 10:35:02 - Median flatfields for ut20170323_geminiN_twiskyflat already generated.
2024-04-04 10:35:02 - Median flatfields for ut20170402_geminiN_twiskyflat already generated.
2024-04-04 10:35:02 - Median flatfields for ut20170408_geminiN_twiskyflat already generated.
2024-04-04 10:35:02 - Median flatfields for ut20170410_geminiN_twiskyflat already generated.
2024-04-04 10:35:02 - Median flatfields for ut20170425_geminiN_twiskyflat already generated.
2024-04-04 10:35:02 - Median flatfields for ut20170507_geminiN_twiskyflat already generated.
2024-04-04 10:35:02 - Median flatfields for ut20170524_geminiN_twiskyflat already generated.
2024-04-04 10:35:02 - Median flatfields for ut20170621_geminiN_twiskyflat already generated.
2024-04-04 10:35:02 - Median flatfields for ut20170625_geminiN_twiskyflat already generated.
2024-04-04 10:35:02 - Median flatfields for ut20170901_geminiN_twiskyflat already generated.
2024-04-04 10:35:02 - Median flatfields for ut20170902_geminiN_twiskyf

2024-04-04 10:35:03 - Median flatfields for ut20150606_geminiS_twiskyflat already generated.
2024-04-04 10:35:03 - Median flatfields for ut20150905_geminiS_twiskyflat already generated.
2024-04-04 10:35:03 - Median flatfields for ut20150911_geminiS_twiskyflat already generated.
2024-04-04 10:35:03 - Median flatfields for ut20151122_geminiS_twiskyflat already generated.
2024-04-04 10:35:03 - Median flatfields for ut20151207_geminiS_twiskyflat already generated.
2024-04-04 10:35:03 - Median flatfields for ut20151226_geminiS_twiskyflat already generated.
2024-04-04 10:35:03 - Median flatfields for ut20160112_geminiS_twiskyflat already generated.
2024-04-04 10:35:03 - Median flatfields for ut20160122_geminiS_twiskyflat already generated.
2024-04-04 10:35:03 - Median flatfields for ut20160204_geminiS_twiskyflat already generated.
2024-04-04 10:35:03 - Median flatfields for ut20160223_geminiS_twiskyflat already generated.
2024-04-04 10:35:03 - Median flatfields for ut20160401_geminiS_twiskyf

>>> Median combination of flatfield frames for Chip 1 complete.
>>> Median combination of flatfield frames for Chip 2 complete.
>>> Median combination of flatfield frames for Chip 3 complete.
2024-04-04 10:36:08 - Median combination of flatfield frames complete.
>>> Starting fpack compression of FITS files in /volumes/Fantom12a/BackupData/gemini/data_calib/GMOS-S_TwilightFlats_20150101_present/GMOS-S_Twilight_Flats_r/ut20240329_geminiS_twiskyflat/procfits_twiskyflat/...
>>> fpack compression of FITS files complete.
2024-04-04 10:36:31 - Processing flatfield files in ut20240329_geminiS_twiskyflat complete.
Done.
